# **Sequential Hyperparameter Search and Sensitivity Analysis of Color Channels**

This notebook implements sequential hyperparameter search, sensitivity analysis of the RGB color channel, and systematic experiment tracking using Weights & Biases (wandb).

Instead of performing a full grid search (computationally expensive), a sequential strategy is used:  
At each stage:
- Only one hyperparameter is varied
- All other hyperparameters remain fixed
- The best-performing value is selected
- That value is fixed before moving to the next stage

This approach significantly reduces computational cost while maintaining controlled experimentation.

In [ ]:
import tensorflow as tf
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import wandb
from wandb.integration.keras import WandbMetricsLogger

from tools import load_images_with_labels, calculate_metrics, evaluate_model

## Load data

In [ ]:
#Image dimensions
width = 540
height = 960

#Path where images are stored and organized by class
path = '../data/burn_images/'

#Channel selection for sensitivity analysis
#['red'], ['green'], ['blue']
#['red','green'], ['red','blue'], ['green','blue']
#['red','green','blue'] or 'bgr'
channels = ['green', 'blue'] 
n_channels = len(channels) #Number of input channels

In [ ]:
#Load images with their corresponding labels using only selected channels
X, y = load_images_with_labels(path=path, channels=channels)
print(X.shape, y.shape)

In [ ]:
#Split the dataset into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

print('Shape of X_train:', X_train.shape)
print('Shape of X_val:', X_val.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_val:', y_val.shape)

## Función para cambiar los hiperparámetros

Esta función recibe un hiperparámetro y una lista de hiperparámetros y se van probando uno por uno en el modelo.  

In [ ]:
#Diccionario con los valores base de los hiperparámetros
base_hyperparams = {
    'conv_layers': 3,
    'filters_layer_1': 32,
    'filters_layer_2': 32,
    'filters_layer_3': 64,
    'kernel_size': 3,
    'strides': 2,
    'dense_layers': 1,
    'dense_units_1': 64,
    'dense_units_2': 64,
    'dense_units_3': 64,
    'dropout_rate': 0.4,
    'batch_size': 32
}

In [ ]:
def cambiar_parametro(diccionario, hiperparametro, lista_hiperparametros):
    '''
    Función que cambia el valor de un hiperparámetro en un diccionario y genera un nombre de ejecución.
    Es decir, modifica un único hiperparámetro mientras mantiene los demás constantes.

    Parámetros:
    - diccionario (dict): Diccionario que contiene los parámetros del modelo.
    - hiperparametro (str): Clave del parámetro que se desea cambiar.
    - lista_hiperparametros (list): Lista de valores que se asignarán al parámetro.

    Yields:
    - tuple: Un nombre de ejecución (str) y el diccionario actualizado (dict).
    Si el parámetro no se encuentra en el diccionario, imprime un mensaje de error y retorna None.
    '''
    for valor in lista_hiperparametros:
        #Se modifica únicamente el hiperparámetro objetivo
        if hiperparametro in diccionario:
            diccionario[hiperparametro] = valor
        else:
            print(f'El hiperparametro {hiperparametro} no se encuentra en el diccionario')
            return None
        
        #Construcción del nombre del experimento
        nombre_run = (f"cl:{diccionario['conv_layers']}, " 
        + f"fl1:{diccionario['filters_layer_1']}, ")
        
        if diccionario['conv_layers'] >= 2: 
            nombre_run = nombre_run + f"fl2:{diccionario['filters_layer_2']}, "
        if diccionario['conv_layers'] >= 3: 
            nombre_run = nombre_run + f"fl3:{diccionario['filters_layer_3']}, "

        nombre_run = (nombre_run + f"ks:{diccionario['kernel_size']}, "
            + f"st:{diccionario['strides']}, "
            + f"dl:{diccionario['dense_layers']}, "
            + f"du1:{diccionario['dense_units_1']}, ")
        
        if diccionario['dense_layers'] >= 2:
            nombre_run = nombre_run + f"du2:{diccionario['dense_units_2']}, "  
        if diccionario['dense_layers'] >= 3:
            nombre_run = nombre_run + f"du3:{diccionario['dense_units_3']}, "  

        nombre_run = (nombre_run + f"dr:{diccionario['dropout_rate']}, "
            + f"bs:{diccionario['batch_size']}")

        yield nombre_run, diccionario

In [ ]:
#Crear generador de nombres y diccionarios
generador_parametros = cambiar_parametro(base_hyperparams, 'batch_size', [32])

### A partir de aquí correr para probar los demás valores de la lista

In [ ]:
nombre_run, parametros = next(generador_parametros)
print(nombre_run)

## CNN model construction

In [ ]:
#First convolutional block (includes input shape)
arquitectura = [layers.Conv2D(parametros["filters_layer_1"], 
                              kernel_size=(parametros["kernel_size"], parametros["kernel_size"]), 
                              strides=(parametros["strides"], parametros["strides"]),
                              input_shape=(height, width, n_channels)),
                layers.BatchNormalization(),
                layers.Activation('relu'),
                layers.MaxPooling2D(pool_size=(2, 2))]

#Remaining convolutional layers (added according to the value of conv_layers)
if parametros['conv_layers'] > 1:
    arquitectura += [layers.Conv2D(parametros["filters_layer_2"], 
                                   kernel_size=(parametros["kernel_size"], parametros["kernel_size"]),
                                   strides=(parametros["strides"], parametros["strides"])),
                    layers.BatchNormalization(),
                    layers.Activation('relu'),
                    layers.MaxPooling2D(pool_size=(2, 2))]

    if parametros['conv_layers'] > 2:
        arquitectura += [layers.Conv2D(parametros["filters_layer_3"], 
                                       kernel_size=(parametros["kernel_size"], parametros["kernel_size"]),
                                       strides=(parametros["strides"], parametros["strides"])),
                        layers.BatchNormalization(),
                        layers.Activation('relu'),
                        layers.MaxPooling2D(pool_size=(2, 2))]
            
#Feature aggregation
arquitectura += [layers.GlobalAveragePooling2D()]

#Fully connected layers (added according to the value of dense_layers)
arquitectura += [layers.Dense(parametros["dense_units_1"]),
                layers.Activation('relu'),
                layers.Dropout(parametros["dropout_rate"])]

if parametros['dense_layers'] > 1:
    arquitectura += [layers.Dense(parametros["dense_units_2"]),
                    layers.Activation('relu'),
                    layers.Dropout(parametros["dropout_rate"])]

    if parametros['dense_layers'] > 2:
        arquitectura += [layers.Dense(parametros["dense_units_3"]),
                        layers.Activation('relu'),
                        layers.Dropout(parametros["dropout_rate"])]

#Output layer: probability of third-degree burn
arquitectura += [layers.Dense(1),
                layers.Activation('sigmoid')]

model = tf.keras.Sequential(arquitectura) #Final model

In [ ]:
#Display the final model architecture
model.summary()

In [ ]:
#Model compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Crear experimento en wandb

In [ ]:
# Se crea el experimento en wandb
configuracion = {'conv_layers': parametros["conv_layers"],
                 'filters_layer_1': parametros["filters_layer_1"],
                 'kernel_size': parametros["kernel_size"],
                 'strides': parametros["strides"],
                 'dense_layers': parametros["dense_layers"],
                 'dense_units_1': parametros["dense_units_1"],
                 'dropout_rate': parametros["dropout_rate"],
                 'batch_size': parametros["batch_size"]}

if parametros["conv_layers"] > 1:
    configuracion['filters_layer_2'] = parametros["filters_layer_2"]
    if parametros["conv_layers"] > 2:
        configuracion['filters_layer_3'] = parametros["filters_layer_3"]

if parametros["dense_layers"] > 1:
    configuracion['dense_units_2'] = parametros["dense_units_2"]
    if parametros["dense_layers"] > 2:
        configuracion['dense_units_3'] = parametros["dense_units_3"]

run = wandb.init(project="CNN_quemaduras_2", 
                 entity="frantorres14",
                 name="_".join(channels), #Poner nombre_run para la búsqueda de hiperparámetros
                 config=configuracion)

config = wandb.config
wandb_logger = WandbMetricsLogger(config)

# Se entrena el modelo

In [ ]:
#Stop training when the model stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, #Wait 10 epochs without improvement before stopping
    restore_best_weights=True #Restore weights from the epoch with the best val_loss
)

#Reduce the learning rate when the model stagnates
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, #Reduce the learning rate by half when there is no improvement
    patience=5, #Wait 5 epochs without improvement before reducing
    min_lr=1e-7 #Minimum allowed learning rate
)

In [ ]:
#Model training
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=config.batch_size,
                    validation_data=(X_val, y_val),
                    callbacks=[WandbMetricsLogger(), early_stopping, reduce_lr])

# Métricas de evaluación

In [ ]:
CM_train, accuracy_train, precision_train, recall_train, f1_train = calculate_metrics(model, X_train, y_train)
evaluate_model(model, X_train, y_train, dataset='Training')

In [ ]:
CM_val, accuracy_val, precision_val, recall_val, f1_val = calculate_metrics(model, X_val, y_val)
evaluate_model(model, X_val, y_val, dataset='Validation')

In [ ]:
# Registrar métricas en wandb
wandb.log({"accuracy_train": accuracy_train,
           "precision_train": precision_train,
           "recall_train": recall_train,
           "f1_train": f1_train,
           "accuracy_val": accuracy_val,
           "precision_val": precision_val,
           "recall_val": recall_val,
           "f1_val": f1_val})

# Termina el experimento
run.finish()

### Hiperparámetros a probar

'conv_layers': [1, 2, 3]  
'filters_layer_k': [16, 32, 64]  
'kernel_size': [3, 5]  
'strides': [1, 2, 3]  
'dense_layers': [1, 2, 3]  
'dense_units_k': [32, 64, 128]  
'dropout_rate': [0.3, 0.4, 0.5]  
'batch_size': [16, 32, 64]  